In [1]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

data = pd.read_csv(r"./Tweets.csv")
data.head(5)

/Users/ethanxie/Documents/VSCode Projects/sentiment/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [2]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["sentiment"], test_size=0.2, stratify=data["sentiment"])
X_train = X_train.fillna("")
X_test = X_test.fillna("")

len(X_train), len(X_test)

(21984, 5497)

In [3]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

encodings_train = tokenizer(
    list(X_train),
    truncation=True,
    padding="max_length",
    max_length=128,
    return_tensors="pt"
)

encodings_test  = tokenizer(
    list(X_test),
    truncation=True,
    padding="max_length",
    max_length=128,
    return_tensors="pt"
)

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(encodings_train["input_ids"], encodings_train["attention_mask"], y_train)
test_dataset = TensorDataset(encodings_test["input_ids"], encodings_test["attention_mask"], y_test)

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(le.classes_))
model.to(device)

Using mps device


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [7]:
N = 100
train_dataloader = DataLoader(train_dataset, batch_size=N)
test_dataloader = DataLoader(test_dataset, batch_size=N)

def train(model, dataloader, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for i, batch in enumerate(dataloader):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 100 == 0:
            loss, current = loss.item(), i * len(batch)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(model, dataloader):
    size = len(dataloader.dataset)
    num_N = len(dataloader)
    
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            test_loss += outputs.loss.item()
            logits = outputs.logits
            correct += (logits.argmax(1) == labels).type(torch.float).sum().item()
    test_loss /= num_N
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [8]:
epochs = 1

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    train(model, train_dataloader, optimizer)
    test(model, test_dataloader)

print("Done!")

Epoch 1
-------------------------------
loss: 0.509765  [    0/21984]
loss: 0.514909  [  300/21984]
loss: 0.402961  [  600/21984]
Test Error: 
 Accuracy: 77.4%, Avg loss: 0.583301 

Done!


In [9]:
model.save_pretrained("./bert-sentiment")
tokenizer.save_pretrained("./bert-sentiment")

('./bert-sentiment/tokenizer_config.json',
 './bert-sentiment/special_tokens_map.json',
 './bert-sentiment/vocab.txt',
 './bert-sentiment/added_tokens.json')